# Tiền xử lý dũ liệu #

## VLSP 2020 (65h) ##

In [1]:
import os 
import librosa

audio_files = []
script_files = []
for root, folders, files in os.walk("./vlsp2020_train_set_02"):
    for file in files:
        args_name = file.split('.')
        if args_name[1] == 'wav':
            audio_files.append(os.path.join(root,file))
        elif args_name[1] == 'txt':
            script_files.append(os.path.join(root,file))

In [1]:
import os
from tqdm import tqdm
import librosa
import pickle

def mappingAudioScript(audio_path, script_path):
    obj = {
            'path': audio_path,
            'array': [],
            'sentence': ""
        }
    try:
        y, sr = librosa.load(audio_path,sr=16000,mono=True)
        obj['array'] = y
        obj['time'] = librosa.get_duration(y=y,sr=sr)
        with open(script_path,encoding='utf-8') as f: 
            sentence = f.readline().strip()
        obj['sentence'] = sentence
        return obj
    except:
        print("not file Script: ",script_path)
        return obj
    
def read_data(Audio_Folder, Script_Folder, audioFormat):
    lst = [] 
    for root, _, files in os.walk(Audio_Folder):
        for name in tqdm(files):
            try:
                if name.split('.')[-1] == audioFormat:
                    audio_path = os.path.join(root,name)
                    args = root.split(os.sep)
                    args[0] = Script_Folder
                    Script_Folder_Path = os.sep.join(args)
                    script_path = os.path.join(Script_Folder_Path,name.split('.')[0]+'.txt')
                    obj = mappingAudioScript(audio_path,script_path)
                    if obj['time'] > 6 and obj['time'] < 30:
                        lst.append(obj)
            except Exception as e:
                pass
    with open('./VLSP.pkl', 'wb') as f:
        pickle.dump(lst, f)

if __name__ == '__main__':
    audioFormat = 'wav'
    Audio_Folder = 'vlsp2020_train_set_02'
    Script_Folder = 'vlsp2020_train_set_02'
    read_data(Audio_Folder,Script_Folder,audioFormat)


100%|██████████| 112854/112854 [23:21<00:00, 80.52it/s] 


In [3]:
#Read *.pkl
import pickle

def read_file_pkl(pkl_file_path):
    with open(pkl_file_path,'rb') as f:
        data = pickle.load(f)
    return data

In [ ]:
data = read_file_pkl("/mnt/wsl/PHYSICALDRIVE0p1/toan/data_speech_to_text/Public dataset/VLSP.pkl")
data

## Vivos (15h) ##

In [ ]:
import os
import librosa 
from tqdm import tqdm

FOLDER_AUDIO = '/mnt/wsl/PHYSICALDRIVE0p1/toan/data_speech_to_text/Public dataset/vivos/data/vivos'
SCRIPT_TRAIN_PATH = '/mnt/wsl/PHYSICALDRIVE0p1/toan/data_speech_to_text/Public dataset/vivos/data/prompts-train.txt'
SCRIPT_TEST_PATH = '/mnt/wsl/PHYSICALDRIVE0p1/toan/data_speech_to_text/Public dataset/vivos/data/prompts-test.txt'

#read_script
audio2script = {}
with open(SCRIPT_TRAIN_PATH, encoding = 'utf-8') as f:
    for row in f:
        data = row.strip().split(" ", 1)
        audio2script[data[0]] = data[1]
        
with open(SCRIPT_TEST_PATH, encoding = 'utf-8') as f:
    for row in f:
        data = row.strip().split(" ", 1)
        audio2script[data[0]] = data[1]

def mappingAudioScript(audio_path, file_name):
    obj = {
            'path': audio_path,
            'array': [],
            'sentence': ""
        }
    try:
        y, sr = librosa.load(audio_path,sr=16000,mono=True)
        obj['array'] = y
        obj['time'] = librosa.get_duration(y=y,sr=sr)
        obj['sentence'] = audio2script.get(file_name,"")
        return obj
    except Exception as e:
        print('error: ',e)
        return obj
lst = [] 
# read audio
for root, _, files in os.walk(FOLDER_AUDIO):
    for file in tqdm(files):
        if file.split('.')[-1] == 'wav':
            audio_path = os.path.join(root,file)
            args = root.split(os.sep)
            obj = mappingAudioScript(audio_path,file.split('.')[0])
            if obj['time'] < 30:
                lst.append(obj)


In [5]:
import pickle
with open('./VIVOS.pkl', 'wb') as f:
    pickle.dump(lst, f)

## FPT (30h) ##

In [11]:
import os
import librosa 
from tqdm import tqdm

FOLDER_AUDIO = '/mnt/wsl/PHYSICALDRIVE0p1/toan/data_speech_to_text/Public dataset/FPT/mp3'
SCRIPT_PATH = '/mnt/wsl/PHYSICALDRIVE0p1/toan/data_speech_to_text/Public dataset/FPT/transcriptAll.txt'


#read_script
audio2script = {}
with open(SCRIPT_PATH, encoding = 'utf-8') as f:
    for row in f:
        data = row.strip().split("|")
        audio2script[data[0]] = data[1]
        
def mappingAudioScript(audio_path, file_name):
    obj = {
            'path': audio_path,
            'array': [],
            'sentence': ""
        }
    try:
        y, sr = librosa.load(audio_path,sr=16000,mono=True)
        obj['array'] = y
        obj['time'] = librosa.get_duration(y=y,sr=sr)
        obj['sentence'] = audio2script.get(file_name,"")
        return obj
    except Exception as e:
        print('error: ',e)
        return obj
lst = [] 

# read audio
for root, _, files in os.walk(FOLDER_AUDIO):
    for file in tqdm(files):
        if file.split('.')[-1] == 'mp3':
            audio_path = os.path.join(root,file)
            args = root.split(os.sep)
            obj = mappingAudioScript(audio_path,file)
            if obj['time'] < 30:
                lst.append(obj)

100%|██████████| 60197/60197 [02:17<00:00, 438.99it/s] 


In [12]:
time = 0
for record in lst:
    time+=record['time']
print(time/3600)

30.163563333333894


In [ ]:
import pickle
with open('./FPT.pkl', 'wb') as f:
    pickle.dump(lst, f)

## AIA ASR (159h) ##

In [1]:
from pymongo import MongoClient
import os
from tqdm import tqdm

#Connect2Database
client = MongoClient("192.168.1.60", 27017)
collection = client['ASR']['ASR_All'] 

In [10]:
result = list(collection.find({"compare": True, "time": {"$lte":30 }}, {'_id':0, 'gold_sentence': 0, 'labeled': 0, 'humman': 0 }))

In [11]:
len(result)

291082

In [12]:
time = 0
for record in result:
    time += record['time']
time/3600

159.74713348960978